In [1]:
import os
import random
import csv
import os
import subprocess
import stanza
from tensorflow.core.example import example_pb2
import tensorflow as tf
import collections
import struct

In [2]:
transcript = './transcripts'
abstract_dir = './abstractive'

In [3]:
abb = []
for e in os.listdir(abstract_dir):
    abb.append(e[:e.find('.')])
train = []
test = []
validate = []
for e in abb:
    num = random.randint(1, 20)
    if num <= 14:
        train.append(e)
    elif num >= 18:
        validate.append(e)
    else:
        test.append(e)

In [4]:
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence

def file_return_str(file):
    f = open(file, "r")
    string = [] 
    while(True):
        line = f.readline()
        if not line:
            break
        string.append(line.strip())
    string = ' '.join(string)
    if string[-1] not in END_TOKENS:
        string = string + "."
    f.close
    return string.lower()

def process(summary, text, nlp):
    s = nlp(summary)
    s_um = []
    for i, sentence in enumerate(s.sentences):
        #s_um.append('<s>')
        for token in sentence.tokens:
            s_um.append(token.text)
        #s_um.append('</s>')
    t = nlp(text)
    t_ex = []
    for i, sentence in enumerate(t.sentences):
        for token in sentence.tokens:
            t_ex.append(token.text)
    article = ' '.join(t_ex)
    abstract = ' '.join(s_um)
    return abstract, article



In [6]:
nlp = stanza.Pipeline(lang='en', processors='tokenize')

2021-06-26 16:31:20 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-26 16:31:20 INFO: Use device: cpu
2021-06-26 16:31:20 INFO: Loading: tokenize
2021-06-26 16:31:20 INFO: Done loading processors!


In [5]:
with open(os.path.join('./out', 'train.bin') , 'wb') as writer:
    for e in train:
        articl = file_return_str(os.path.join(transcript, e + '.transcript.txt'))
        abstrac = file_return_str(os.path.join(abstract_dir, e + '.abssumm.txt'))
        abstract, article = process(abstrac, articl, nlp)
        tf_example = example_pb2.Example()
        tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
        tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode()])
        tf_example_str = tf_example.SerializeToString()
        str_len = len(tf_example_str)
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, tf_example_str))
    

2021-06-26 16:30:55 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-26 16:30:55 INFO: Use device: cpu
2021-06-26 16:30:55 INFO: Loading: tokenize
2021-06-26 16:30:55 INFO: Done loading processors!


KeyboardInterrupt: 

In [ ]:
with open(os.path.join('./out', 'val.bin') , 'wb') as writer:
    for e in validate:
        articl = file_return_str(os.path.join(transcript, e + '.transcript.txt'))
        abstrac = file_return_str(os.path.join(abstract_dir, e + '.abssumm.txt'))
        abstract, article = process(abstrac, articl, nlp)
        tf_example = example_pb2.Example()
        tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
        tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode()])
        tf_example_str = tf_example.SerializeToString()
        str_len = len(tf_example_str)
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, tf_example_str))

In [7]:
with open(os.path.join('./out', 'test.bin') , 'wb') as writer:
    for e in test:
        articl = file_return_str(os.path.join(transcript, e + '.transcript.txt'))
        abstrac = file_return_str(os.path.join(abstract_dir, e + '.abssumm.txt'))
        abstract, article = process(abstrac, articl, nlp)
        tf_example = example_pb2.Example()
        tf_example.features.feature['article'].bytes_list.value.extend([article.encode()])
        tf_example.features.feature['abstract'].bytes_list.value.extend([abstract.encode()])
        tf_example_str = tf_example.SerializeToString()
        str_len = len(tf_example_str)
        writer.write(struct.pack('q', str_len))
        writer.write(struct.pack('%ds' % str_len, tf_example_str))